In [1]:
import xml.etree.ElementTree as et

from glob import glob
import json
import pandas as pd
import re
from tqdm import tqdm

import xml.etree.ElementTree as et 

In [2]:
def xml_to_csv(xml):
    except_list = []
    df_cols=["category","name","cn"]
    df = pd.DataFrame(columns = df_cols)
    cnt = 0
    for file in xml:
        try:
            parser = et.XMLParser(encoding="UTF-8")
            xtree = et.parse(file,parser=parser)
            xroot = xtree.getroot()
            rows=[]

            for node in xroot:
                s_category=node.find("category").text if node is not None else None
                s_name=node.find("name").text if node is not None else None
                s_cn=node.find("cn").text if node is not None else None
                
                rows.append({"category":s_category, "name":s_name, "cn":s_cn})
            
            out_df = pd.DataFrame(rows,columns=df_cols)
            df = pd.concat([df,out_df])
        except:
            cnt += 1
            except_list.append(file)
    return df,except_list
train_xml_good_list = glob('../Data/Train/Document/01.유리/*.xml')
train_xml_bad_list = glob('../Data/Train/Document/02.불리/*.xml')
test_xml_good_list = glob('../Data/Test/Document/01.유리/*.xml')
test_xml_bad_list = glob('../Data/Test/Document/02.불리/*.xml')

train_xml_good_list = sorted(train_xml_good_list)
train_xml_bad_list = sorted(train_xml_bad_list)
test_xml_good_list = sorted(test_xml_good_list)
test_xml_bad_list = sorted(test_xml_bad_list)

train_xml_good,trg_list = xml_to_csv(train_xml_good_list)
train_xml_bad,trb_list = xml_to_csv(train_xml_bad_list)
test_xml_good,teg_list = xml_to_csv(test_xml_good_list)
test_xml_bad,teb_list = xml_to_csv(test_xml_bad_list)

train_xml_good['ad_label'] = 1
train_xml_bad['ad_label'] = 2
test_xml_good['ad_label'] = 1
test_xml_bad['ad_label'] = 2

print(train_xml_good.shape[0]+train_xml_bad.shape[0]+test_xml_good.shape[0]+test_xml_bad.shape[0])

train_xml_good.reset_index(inplace=True,drop=True)
train_xml_bad.reset_index(inplace=True,drop=True)
test_xml_good.reset_index(inplace=True,drop=True)
test_xml_bad.reset_index(inplace=True,drop=True)

9000


In [3]:
def xml_name_preprocess(df):
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Train/Document/01.유리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Train/Document/02.불리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Test/Document/01.유리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Test/Document/02.불리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('.xml',''))
    df['name'] = df['name'].apply(lambda x: x.replace('_가공',''))
    df['name'] = df['name'].apply(lambda x: x.replace(' ',''))
    df['name'] = df['name'].apply(lambda x: x.strip())
    return df

train_xml_good_name = pd.DataFrame(train_xml_good_list,columns=['name'])
train_xml_good_name = xml_name_preprocess(train_xml_good_name)
print(train_xml_good_name.nunique())

train_xml_bad_name = pd.DataFrame(train_xml_bad_list,columns=['name'])
train_xml_bad_name = xml_name_preprocess(train_xml_bad_name)
print(train_xml_bad_name.nunique())

test_xml_good_name = pd.DataFrame(test_xml_good_list,columns=['name'])
test_xml_good_name = xml_name_preprocess(test_xml_good_name)
print(test_xml_good_name.nunique())

test_xml_bad_name = pd.DataFrame(test_xml_bad_list,columns=['name'])
test_xml_bad_name = xml_name_preprocess(test_xml_bad_name)
print(test_xml_bad_name.nunique())

train_xml_good['name'] = train_xml_good_name['name']
train_xml_bad['name'] = train_xml_bad_name['name']
test_xml_good['name'] = test_xml_good_name['name']
test_xml_bad['name'] = test_xml_bad_name['name']

name    5600
dtype: int64
name    2400
dtype: int64
name    700
dtype: int64
name    300
dtype: int64


In [4]:
# json_to_csv
def json_to_csv(files):
    df_cols=["clauseArticle","dvAntageous"]
    df = pd.DataFrame(columns = df_cols)
        
    for file in files:   
        with open(file) as f:
            json_data = json.load(f)
            
            clauseArticle = json_data['clauseArticle']
            dvAntageous = json_data['dvAntageous']
            row = []
            row.append({"clauseArticle":clauseArticle, "dvAntageous":dvAntageous})
        out_df = pd.DataFrame(row,columns=df_cols)
        df = pd.concat([df,out_df])
        
    file_name = pd.DataFrame(files)
    file_name[0] = file_name[0].apply(lambda x: x.split('/')[-1])
    df.reset_index(inplace=True)
    df = pd.concat([df,file_name],axis=1)
    df.drop(['index'],inplace=True,axis=1)
    df.columns = ['clauseArticle','ad_label','name']
    return df

train_json_good_list = glob('../Data/Train/Label/01.유리/*.json')
train_json_bad_list = glob('../Data/Train/Label/02.불리/*.json')
test_json_good_list = glob('../Data/Test/Label/01.유리/*.json')
test_json_bad_list = glob('../Data/Test/Label/02.불리/*.json')

train_json_good_list = sorted(train_json_good_list)
train_json_bad_list = sorted(train_json_bad_list)
test_json_good_list = sorted(test_json_good_list)
test_json_bad_list = sorted(test_json_bad_list)


train_json_good = json_to_csv(train_json_good_list)
train_json_bad = json_to_csv(train_json_bad_list)
test_json_good = json_to_csv(test_json_good_list)
test_json_bad = json_to_csv(test_json_bad_list)

train_json_good.reset_index(inplace=True,drop=True)
train_json_bad.reset_index(inplace=True,drop=True)
test_json_good.reset_index(inplace=True,drop=True)
test_json_bad.reset_index(inplace=True,drop=True)

In [5]:
def json_name_preprocess(df):
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Train/Label/01.유리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Train/Label/02.불리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Test/Label/01.유리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('../Data/Test/Label/02.불리\\',''))
    df['name'] = df['name'].apply(lambda x: x.replace('_가공.json',''))
    df['name'] = df['name'].apply(lambda x: x.replace(' ',''))
    df['name'] = df['name'].apply(lambda x: x.strip())
    return df

train_json_good_name = pd.DataFrame(train_json_good_list,columns=['name'])
train_json_good_name = json_name_preprocess(train_json_good_name)
print(train_json_good_name.nunique())

train_json_bad_name = pd.DataFrame(train_json_bad_list,columns=['name'])
train_json_bad_name = json_name_preprocess(train_json_bad_name)
print(train_json_bad_name.nunique())

test_json_good_name = pd.DataFrame(test_json_good_list,columns=['name'])
test_json_good_name = json_name_preprocess(test_json_good_name)
print(test_json_good_name.nunique())

test_json_bad_name = pd.DataFrame(test_json_bad_list,columns=['name'])
test_json_bad_name = json_name_preprocess(test_json_bad_name)
print(test_json_bad_name.nunique())

train_json_good['name'] = train_json_good_name['name']
train_json_bad['name'] = train_json_bad_name['name']
test_json_good['name'] = test_json_good_name['name']
test_json_bad['name'] = test_json_bad_name['name']

name    5600
dtype: int64
name    2400
dtype: int64
name    700
dtype: int64
name    300
dtype: int64


In [6]:
train_json_good['ad_label'] = train_json_good['ad_label'].apply(lambda x: int(x))
train_json_bad['ad_label'] = train_json_bad['ad_label'].apply(lambda x: int(x))
test_json_good['ad_label'] = test_json_good['ad_label'].apply(lambda x: int(x))
test_json_bad['ad_label'] = test_json_bad['ad_label'].apply(lambda x: int(x))

train_good = pd.merge(train_xml_good,train_json_good,on=['name','ad_label'],how='inner')
train_bad = pd.merge(train_xml_bad,train_json_bad,on=['name','ad_label'],how='inner')
test_good = pd.merge(test_xml_good,test_json_good,on=['name','ad_label'],how='inner')
test_bad = pd.merge(test_xml_bad,test_json_bad,on=['name','ad_label'],how='inner')
print(train_good.shape)
print(train_bad.shape)
print(test_good.shape)
print(test_bad.shape)

(5600, 5)
(2400, 5)
(700, 5)
(300, 5)


In [7]:
df = pd.concat([train_good,train_bad,test_good,test_bad])
df.reset_index(inplace=True,drop=True)
df = df[['category','name','ad_label','cn','clauseArticle']]
df.columns = ['category','name','ad_label','cn','summary']

df['category'] = df['category'].apply(lambda x: x.replace('\n',''))
df['category'] = df['category'].apply(lambda x: x.replace('\t',''))
df['category'] = df['category'].apply(lambda x: x.replace('.pdf',''))

In [8]:
df.shape

(9000, 5)

In [9]:
df.head()

,category,name,ad_label,cn,summary
0,24. 개인정보취급방침,001_개인정보취급방침,1,\n\t\t\n\n\n\n\n\n제 1 조 개인정보의 처리 목적\n회사는 다음의 목...,[제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이...
1,27. 결혼정보서비스,001_결혼정보서비스,1,\n\t\t\n\n\n000 \n\n \n\n제1조(목적 등) \n\n1. 본 약...,[제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...
2,30.보증,001_보증,1,\n\t\t\n\n \n\n \n \n\n신용보증약정서 \n \n\n000 귀중\n...,[제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증...
3,31. 사이버몰,001_사이버몰,1,\n\t\t\n\n \n제1조(목적) \n이 약관은 000 회사(전자상거래 사업자)...,[제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 ...
4,12. 상해보험,001_상해보험,1,\n\t\t\n\n \n\n00000 \n\n약관 \n \n\n제1관 목적 및 ...,[제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 ...


In [10]:
df.to_csv('./preprocessed_data/data.csv')